In [45]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime
from bs4 import BeautifulSoup

In [46]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['BOF2024_crawling']
collection = db['naver_blog_crawling']

In [47]:
# 웹드라이버 초기화 및 네이버 블로그 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://section.blog.naver.com/")
driver.maximize_window()

# 3초간 대기
time.sleep(3)

In [48]:
# 검색어 입력 요소 찾기
search_box = driver.find_element(By.NAME, 'sectionBlogQuery')

# 검색어 입력
search_box.send_keys('부산원아시아페스티벌')

# 3초간 대기 후 엔터 키 입력
time.sleep(3)
search_box.send_keys(Keys.RETURN)

In [49]:
time.sleep(3)

try:
    SCROLL_PAUSE_TIME = 1.5
    current_page = 1
    max_page = 572  # 최대 페이지 수
    all_data = []

    while current_page <= max_page:
        # 맨 아래까지 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        time.sleep(2)

        # 모든 포스트 요소 가져오기
        post_elements = driver.find_elements(By.CSS_SELECTOR, 'div.list_search_post')
        
        for post in post_elements:
            try:
                # 제목 요소 찾기
                title_element = post.find_element(By.CSS_SELECTOR, 'span.title')
                html_content = title_element.get_attribute('innerHTML')
                soup = BeautifulSoup(html_content, 'html.parser')
                full_text = soup.get_text(separator=' ').strip()

                # URL 요소 찾기
                url_tag = post.find_element(By.CSS_SELECTOR, 'div.desc a.desc_inner')
                url_text = url_tag.get_attribute('href').strip()

                # 블로그 이름 요소 찾기
                blog_name_span = post.find_element(By.CSS_SELECTOR, 'span.name_blog')
                blog_name_text = blog_name_span.text.strip()

                # 내용 요약 요소 찾기
                contents_tag = post.find_element(By.CSS_SELECTOR, 'a.text')
                contents_text = contents_tag.text.strip()

                # 업로드 날짜 요소 찾기
                date_span = post.find_element(By.CSS_SELECTOR, 'span.date')
                date_text = date_span.text.strip()

                # 현재 날짜 가져오기
                current_date = datetime.now().strftime("%Y.%m.%d.")

                # 날짜 형식 조정 (올바른 yyyy.mm.dd 형식으로)
                if any(unit in date_text for unit in ["분 전", "시간 전"]):
                    formatted_date = current_date
                else:
                    # 날짜 정보에서 공백 제거 및 점(.)을 기준으로 분리
                    date_parts = date_text.replace(" ", "").split('.')
                    
                    # 월과 일이 한 자리 숫자인 경우 두 자리로 변환
                    year = date_parts[0]
                    month = date_parts[1].zfill(2) if len(date_parts[1]) == 1 else date_parts[1]
                    day = date_parts[2].zfill(2) if len(date_parts[2]) == 1 else date_parts[2]

                    # 올바른 형식으로 날짜 포맷
                    formatted_date = f"{year}.{month}.{day}."

                # 모든 데이터가 있을 경우 MongoDB에 적재
                if full_text and url_text and blog_name_text and contents_text and date_text:
                    data = {
                        "제목": full_text,
                        "URL": url_text,
                        "블로그이름": blog_name_text,
                        "블로그요약": contents_text,
                        "업로드 날짜": formatted_date,
                        "검색키워드": "부산원아시아페스티벌"  # 검색 키워드를 지정 
                    }
                    all_data.append(data)

            except Exception as e:
                # 필요한 데이터가 없는 경우 해당 포스트를 건너뜁니다.
                continue

        # 다음 페이지로 이동
        try:
            if current_page % 10 == 0:
                next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.button_next")))
                next_button.click()
            else:
                next_page_link = driver.find_element(By.CSS_SELECTOR, f"a.item[aria-label='{current_page + 1}페이지']")
                next_page_link.click()
            current_page += 1
            time.sleep(2)  # 페이지 로딩 시간 대기
        except Exception as e:
            print(f"마지막 페이지 {current_page}에 도달했습니다. 더 이상 다음 페이지가 없습니다.")
            print(f"Exception: {e}")
            break

        time.sleep(1)

    # 데이터 적재 진행 
    for data in tqdm(all_data, desc='MongoDB에 데이터 적재 중'):
        collection.insert_one(data)

finally:
    driver.quit()
    client.close()

C:\Users\123\AppData\Local\Temp\ipykernel_10204\3210467420.py:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
MongoDB에 데이터 적재 중: 100%|██████████| 3838/3838 [00:36<00:00, 104.69it/s]
